In [1]:
import tensorflow as tf
from tensorflow import keras
import mlflow
import numpy as np
import io
import time
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from utils import *

2024-08-21 01:20:29.376390: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-21 01:20:29.494408: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-21 01:20:29.522609: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
def custom_loss(fk_weight, fk_loss_type, exp_scale=1.0, huber_delta=0.01):
    def loss_fn(y_true, y_pred):
        # Mean Squared Error for joint angles
        angle_loss = tf.keras.losses.Huber(delta=huber_delta)(y_true, y_pred)
        
        # Forward kinematics loss
        fk_true = forward_kinematics_tf(y_true)
        fk_pred = forward_kinematics_tf(y_pred)
        
        if fk_loss_type == 'exponential':
            fk_diff = tf.reduce_sum(tf.square(fk_true - fk_pred), axis=-1)
            fk_loss = tf.reduce_mean(1 - tf.exp(-exp_scale * fk_diff))
        elif fk_loss_type == 'huber':
            fk_loss = tf.keras.losses.Huber(delta=huber_delta)(fk_true, fk_pred)
        elif fk_loss_type == 'linear':
            fk_loss = tf.reduce_mean(tf.abs(fk_true - fk_pred))
        elif fk_loss_type == 'mae':
            fk_loss = tf.keras.losses.mae(fk_true, fk_pred)
        else:
            raise ValueError(f"Unsupported forward kinematics loss type: {fk_loss_type}")
        
        # Combine losses
        total_loss = angle_loss + fk_weight * fk_loss
        return total_loss
    return loss_fn

def create_model(config):
    model = keras.Sequential([
        keras.layers.Dense(128, input_shape=(3,)),
        keras.layers.Activation('relu'),
        keras.layers.Dense(256),
        keras.layers.Activation('relu'),
        keras.layers.Dense(256),
        keras.layers.Activation('relu'),
        keras.layers.Dense(128),
        keras.layers.Activation('relu'),
        keras.layers.Dense(3, activation='tanh')
    ])
    
    model.compile(optimizer='adam', loss=custom_loss(
        config['fk_weight'], 
        config['fk_loss_type'],
        exp_scale=config.get('exp_scale', 1.0),
        huber_delta=config.get('huber_delta', 1.0)
    ))
    return model

def run_single_experiment(config):
    (train_inputs, train_outputs), (test_inputs, test_outputs), input_mean, input_std = load_and_preprocess_data()
    
    valid_inputs, test_inputs, valid_outputs, test_outputs = train_test_split(
        test_inputs, test_outputs, test_size=0.5, random_state=42
    )
    
    mlflow.set_experiment(config['experiment_name'])
    
    with mlflow.start_run(run_name=config['model_name']):
        mlflow.log_params(config)
        
        steps_per_epoch = len(train_inputs) // config['batch_size']
        total_steps = steps_per_epoch * config['epochs']
        warmup_steps = int(0.1 * total_steps)
        
        model = create_model(config)
        
        model_summary = io.StringIO()
        model.summary(print_fn=lambda x: model_summary.write(x + '\n'))
        mlflow.log_text(model_summary.getvalue(), "model_summary.txt")
        
        lr_scheduler = CosineDecayWithWarmupCallback(
            config['initial_learning_rate'],
            warmup_steps,
            total_steps
        )
        verbose_logging = VerboseLoggingCallback()
        lr_logger = LearningRateLogger()
        
        callbacks = [lr_scheduler, verbose_logging, lr_logger]
        
        callback_names = [callback.__class__.__name__ for callback in callbacks]
        mlflow.log_param("callbacks", ", ".join(callback_names))
        
        start_time = time.time()
        history = model.fit(
            train_inputs, train_outputs,
            epochs=config['epochs'],
            batch_size=config['batch_size'],
            validation_data=(valid_inputs, valid_outputs),
            callbacks=callbacks
        )
        training_time = time.time() - start_time
        
        for epoch, (loss, val_loss) in enumerate(zip(history.history['loss'], history.history['val_loss'])):
            mlflow.log_metric("train_loss", loss, step=epoch)
            mlflow.log_metric("val_loss", val_loss, step=epoch)
        
        mlflow.log_metric("training_time", training_time)
        
        mlflow.tensorflow.log_model(model, "model")
        
        mlflow.log_text(verbose_logging.get_output(), "training_output.txt")
        
        errors, true_xyz, predicted_xyz = evaluate_model(model, test_inputs, test_outputs, input_mean, input_std)
        
        mlflow.log_metric("mean_error", np.mean(errors))
        mlflow.log_metric("median_error", np.median(errors))
        mlflow.log_metric("90th_percentile_error", np.percentile(errors, 90))
        mlflow.log_metric("max_error", np.max(errors))
        
        true_vs_pred_plot_path = f"./Figures/Regularization/{config['model_name']}_true_vs_predicted.png"
        plot_true_vs_predicted(true_xyz, predicted_xyz, f"{config['model_name']} Model: True vs Predicted", save_path=true_vs_pred_plot_path)
        mlflow.log_artifact(true_vs_pred_plot_path)
        
        error_dist_plot_path = f"./Figures/Regularization/{config['model_name']}_error_distribution.png"
        plot_error_distribution(errors, f"{config['model_name']} Model: Error Distribution", save_path=error_dist_plot_path)
        mlflow.log_artifact(error_dist_plot_path)
        
        print(f"\n{config['model_name']} Model:")
        print(f"Mean Error: {np.mean(errors):.4f}")
        print(f"Median Error: {np.median(errors):.4f}")
        print(f"90th Percentile Error: {np.percentile(errors, 90):.4f}")
        print(f"Max Error: {np.max(errors):.4f}")
        print(f"Training Time: {training_time:.2f} seconds")
        
        return {
            'model': model,
            'history': history,
            'errors': errors,
            'true_xyz': true_xyz,
            'predicted_xyz': predicted_xyz,
            'training_time': training_time
        }

In [4]:
# Create configurations
configs = []
fk_weights = [10, 20]
fk_loss_types = ['exponential', 'huber', 'linear', 'mae']
learning_rates = [0.001, 0.01, 0.05]

# Parameters specific to certain loss types
exp_scales = [0.1, 0.5, 2.0]
huber_deltas = [0.01, 0.05]

for fk_weight in fk_weights:
    for fk_loss_type in fk_loss_types:
        for learning_rate in learning_rates:
            if fk_loss_type == 'exponential':
                for exp_scale in exp_scales:
                    config = {
                        "model_name": f"Model_FKWeight_{fk_weight}_ExpLoss_Scale_{exp_scale}_LR_{learning_rate}",
                        "fk_weight": fk_weight,
                        "fk_loss_type": fk_loss_type,
                        "exp_scale": exp_scale,
                        "epochs": 100,
                        "initial_learning_rate": learning_rate,
                        "batch_size": 65536*2,
                        "experiment_name": "Inverse Kinematics Multi-Loss"
                    }
                    configs.append(config)
            elif fk_loss_type == 'huber':
                for huber_delta in huber_deltas:
                    config = {
                        "model_name": f"Model_FKWeight_{fk_weight}_HuberLoss_Delta_{huber_delta}_LR_{learning_rate}",
                        "fk_weight": fk_weight,
                        "fk_loss_type": fk_loss_type,
                        "huber_delta": huber_delta,
                        "epochs": 100,
                        "initial_learning_rate": learning_rate,
                        "batch_size": 65536*2,
                        "experiment_name": "Inverse Kinematics Multi-Loss"
                    }
                    configs.append(config)
            else:  # linear and mae
                config = {
                    "model_name": f"Model_FKWeight_{fk_weight}_{fk_loss_type.capitalize()}Loss_LR_{learning_rate}",
                    "fk_weight": fk_weight,
                    "fk_loss_type": fk_loss_type,
                    "epochs": 100,
                    "initial_learning_rate": learning_rate,
                    "batch_size": 65536*2,
                    "experiment_name": "Inverse Kinematics Multi-Loss"
                }
                configs.append(config)

# Print the total number of configurations
print(f"Total number of configurations: {len(configs)}")

Total number of configurations: 42


In [5]:
# Run experiments
for config in configs:
    run_single_experiment(config)

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/mlflow/store/tracking/file_store.py", line 317, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "/usr/local/lib/python3.8/dist-packages/mlflow/store/tracking/file_store.py", line 410, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/usr/local/lib/python3.8/dist-packages/mlflow/store/tracking/file_store.py", line 1341, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "/usr/local/lib/python3.8/dist-packages/mlflow/store/tracking/file_store.py", line 1334, in _read_helper
    result = read_yaml(root, file_name)
  File "/usr/local/lib/python3.8/dist-packages/mlflow/utils/file_utils.py", line 309, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' does not exist.")
mlflow.exceptions.MissingConfigException: Yaml file '/tf/workdir/mlruns/mlruns/meta.yaml' does not

Epoch 1/100
13/69 [====>.........................] - ETA: 0s - loss: 2.0765

2024-08-21 01:22:02.814103: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


69/69 [==============================] - 3s 18ms/step - loss: 1.6457 - val_loss: 0.8471
Epoch 2/100
69/69 [==============================] - 1s 15ms/step - loss: 0.5057 - val_loss: 0.3203
Epoch 3/100
69/69 [==============================] - 1s 15ms/step - loss: 0.2437 - val_loss: 0.1794
Epoch 4/100
69/69 [==============================] - 1s 15ms/step - loss: 0.1447 - val_loss: 0.1174
Epoch 5/100
69/69 [==============================] - 1s 15ms/step - loss: 0.1053 - val_loss: 0.0956
Epoch 6/100
69/69 [==============================] - 1s 15ms/step - loss: 0.0909 - val_loss: 0.0879
Epoch 7/100
69/69 [==============================] - 1s 15ms/step - loss: 0.0841 - val_loss: 0.0826
Epoch 8/100
69/69 [==============================] - 1s 15ms/step - loss: 0.0797 - val_loss: 0.0773
Epoch 9/100
69/69 [==============================] - 1s 15ms/step - loss: 0.0774 - val_loss: 0.0749
Epoch 10/100
69/69 [==============================] - 1s 15ms/step - loss: 0.0758 - val_loss: 0.0733
Epoch 11/10

Epoch 40/100
69/69 [==============================] - 1s 15ms/step - loss: 0.0527 - val_loss: 0.0528
Epoch 41/100
69/69 [==============================] - 1s 15ms/step - loss: 0.0528 - val_loss: 0.0523
Epoch 42/100
69/69 [==============================] - 1s 15ms/step - loss: 0.0525 - val_loss: 0.0530
Epoch 43/100
69/69 [==============================] - 1s 15ms/step - loss: 0.0524 - val_loss: 0.0527
Epoch 44/100
69/69 [==============================] - 1s 15ms/step - loss: 0.0523 - val_loss: 0.0525
Epoch 45/100
69/69 [==============================] - 1s 15ms/step - loss: 0.0522 - val_loss: 0.0521
Epoch 46/100
69/69 [==============================] - 1s 15ms/step - loss: 0.0522 - val_loss: 0.0527
Epoch 47/100
69/69 [==============================] - 1s 15ms/step - loss: 0.0521 - val_loss: 0.0520
Epoch 48/100
69/69 [==============================] - 1s 15ms/step - loss: 0.0519 - val_loss: 0.0522
Epoch 49/100
69/69 [==============================] - 1s 15ms/step - loss: 0.0520 - val_los

...

Epoch 79/100
69/69 [==============================] - 1s 15ms/step - loss: 0.1788 - val_loss: 0.1796
Epoch 80/100
69/69 [==============================] - 1s 16ms/step - loss: 0.1797 - val_loss: 0.1834
Epoch 81/100
69/69 [==============================] - 1s 15ms/step - loss: 0.1762 - val_loss: 0.1761
Epoch 82/100
69/69 [==============================] - 1s 15ms/step - loss: 0.1753 - val_loss: 0.1779
Epoch 83/100
69/69 [==============================] - 1s 15ms/step - loss: 0.1749 - val_loss: 0.1793
Epoch 84/100
69/69 [==============================] - 1s 15ms/step - loss: 0.1750 - val_loss: 0.1742
Epoch 85/100
69/69 [==============================] - 1s 16ms/step - loss: 0.1707 - val_loss: 0.1729
Epoch 86/100
69/69 [==============================] - 1s 15ms/step - loss: 0.1722 - val_loss: 0.1716
Epoch 87/100
69/69 [==============================] - 1s 15ms/step - loss: 0.1690 - val_loss: 0.1712
Epoch 88/100
69/69 [==============================] - 1s 15ms/step - loss: 0.1721 - val_los

2024/08/21 02:44:38 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /tmp/tmpcaws98ld/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmpcaws98ld/model/data/model/assets
2024/08/21 02:44:42 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpcaws98ld/model, flavor: tensorflow). Fall back to return ['tensorflow==2.10.1']. Set logging level to DEBUG to see the full traceback. 
2024/08/21 02:44:42 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


8/8 [==============================] - 0s 3ms/step

Model_FKWeight_20_MaeLoss_LR_0.05 Model:
Mean Error: 0.0162
Median Error: 0.0123
90th Percentile Error: 0.0312
Max Error: 3.3078
Training Time: 107.55 seconds
